# CLASE 5.1: UNA INTRODUCCIÓN A LA LIBRERÍA <font color="mediumorchid">POLARS</font>.
---

## Introducción.
En las primeras dos secciones de la asignatura, nos embarcamos en un viaje cuyo objetivo era aprender a hacer un análisis de datos eficiente, eficaz y efectivo utilizando el lenguaje de programación Python. Para ello, nos abocamos a aprender en detalle el manejo de estructuras de datos en formato de arreglos y DataFrames mediante el uso de las librerías **<font color="mediumorchid">Numpy</font>** y **<font color="mediumorchid">Pandas</font>**. La combinación de estas librerías nos ha permitodo construir una base sólida para la manipulación correcta de datos estructurados sin necesitar mayores ayudas de otras librerías en un especto de uso común, prescindiendo de gráficos de demasiada complejidad o análisis muy avanzados.

No obstante lo anterior, en esta sección retomaremos el tema relativo al análisis de datos estructurados, aprendiendo en esta ocasión el uso de una librería más bien nueva en el mundo de Python, llamada **<font color="mediumorchid">Polars</font>**, y cuyo objetivo se define en su [documentación](https://pola-rs.github.io/polars-book/user-guide/) como el *proveernos de una librería "extremadamente rápida" para el manejo de DataFrames"*, y que cumpla con:

- Aprovechar la potencia de todos los nucles de procesamiento en nuestro computador.
- Optimizar consultas (queries) de datos a fin de reducir las localizaciones de memoria innecesarias.
- Manipular datasets de mayor tamaño que el disponible en nuestra memoria RAM.
- Disponer de una interfaz consistente y de fácil uso.
- Tener un esquema estricto que requiera que conozcamos los tipos de datos antes de ejecutar consultas.

**<font color="mediumorchid">Polars</font>** es una librería escrita en el lenguaje de programación Rust, conocido por su alta rapidez de desempeño, comparable a lenguajes de gran poder como C y C++. Además, se trata de una librería que nos entrega control completo algunas partes críticas de un *motor de queries* relativas al desempeño de las consultas y, por consiguiente, de su tiempo de ejecución.

La razón principal por la cual aprenderemos a utilizar **<font color="mediumorchid">Polars</font>** es, por tanto, su fantástica capacidad de manejar DataFrames extreamdamente grandes sin tiempos de ejecución exageradamente grandes. Esto no es casual, ya que como vimos previamente, **<font color="mediumorchid">Pandas</font>** tiene algunos problemas de rendimiento que escalan con el tamaño de los DataFrames que queremos manipular y, además, tiene la *mala constumbre* de llevarse pésimo con quienes deseen paralelizar su trabajo en distintos núcles de un procesador. Por esta razón, haremos el esfuerzo de aprender *casi* lo mismo que aprendimos en **<font color="mediumorchid">Pandas</font>** usando **<font color="mediumorchid">Polars</font>**, con las correspondientes comparaciones, aprovechando que, como en **<font color="mediumorchid">Pandas</font>**, el objeto que más nos va a interesar manipular en **<font color="mediumorchid">Polars</font>** es precisamente el `DataFrame`.

**<font color="mediumorchid">Polars</font>** puede instalarse fácilmente desde el índice de paquetes de Python usando la instrucción

    pip install polars

Y, en adelante, cada vez que importemos **<font color="mediumorchid">Polars</font>** para su uso, lo haremos mediante el alias `pl`. También importaremos **<font color="mediumorchid">Pandas</font>**, a fin de hacer algunas comparaciones en tiempos de ejecución, y **<font color="mediumorchid">Numpy</font>**, para inicializar estructuras desde cero con valores aleatorizados. Finalmente, usaremos la implementación existente de Arrow en Python conocida como **<font color="mediumorchid">PyArrow</font>**, y que nos permitirá hacer cambios de formato en las estructuras de datos de **<font color="mediumorchid">Polars</font>** a otros más familiares, incluso series y DataFrames de **<font color="mediumorchid">Pandas</font>**. Para ello, instalaremos esta librería desde el índice de paquetes de Python como sigue:

    pip install pyarrow

Y ya podemos comenzar a hacer las correspondientes importaciones:

In [1]:
import numpy as np
import pandas as pd
import polars as pl

## Tipos de datos en **<font color="mediumorchid">Polars</font>**.
**<font color="mediumorchid">Polars</font>**, a diferencia de las versiones antiguas de **<font color="mediumorchid">Pandas</font>** (anteriores a la 2.0.0), hace uso de tipos de datos que son propios de un framework conocido como **Apache Arrow**, y que corresponde a su vez a una plataforma de desarrollo para el desarrollo de procesos de analítica en la memoria de nuestro computador de forma *ultra-rápida*. Esto permite que el procesamiento de datos en **<font color="mediumorchid">Polars</font>** sea extremadamente eficiente, aunque mayormente orientado a datos numéricos.

La Tabla (1.1) detalla algunos de los tipos de datos propios de <font color="mediumorchid">Polars</font>.

</p> <p style="text-align: center;">Tabla (1.1): Tipos de datos usados por <font color="mediumorchid">Polars</font> (la mayoría heredados de Arrow)</p>

| Grupo | Tipo | Comentarios |
| :---- | :--- | :---------- |
| Datos numéricos | `Int8` | Entero de precisión 8 bits. |
| Datos numéricos | `Int16` | Entero de precisión 16 bits. |
| Datos numéricos | `Int32` | Entero de precisión 32 bits. |
| Datos numéricos | `Int64` | Entero de precisión 64 bits. |
| Datos numéricos | `Float32` | Número de punto flotante de precisión 32 bits. |
| Datos numéricos | `Float64` | Número de punto flotante de precisión 64 bits. |
| Datos anidados  | `Struct` | Un arreglo de tipo `Struct` se representa por medio de un objetio de tipo `Vec<Series>` y es útil para empaquetar múltiples valores (en general, heterogéneos) |
| Datos anidados  | `List` | Un arreglo de tipo `List` contiene un *arreglo hijo* que contiene los valores de la lista correspondiente. |
| Datos de tiempo | `Date` | Representación de una fecha que, tras bambalinas, se encodea como un entero con precisión de 32 bits. Está limitada a representar días como mínima únidad de tiempo. |
| Datos de tiempo | `Datetime` | Representación de una fecha que, tras bambalinas, se encodea como un entero con precisión de 64 bits. Es capaz de representar microsegundos como mínima únidad de tiempo. |
| Otros | `Boolean` | Valores Booleanos, representados efectivamente como un bit. |
| Otros | `Utf8` | Data de tipo string. |
| Otros | `Binary` | Data almacenada como bytes. |
| Otros | `Object` | Un tipo de dato con soporte limitado que puede representar virtualmente *cualquier cosa*. |
| Otros | `Categorical` | Un conjunto de strings codificados como valores categóricos (comúnmente ordinales). |



## Series en **<font color="mediumorchid">Polars</font>**.
De la misma forma que ocurre en **<font color="mediumorchid">Pandas</font>**, una serie de **<font color="mediumorchid">Polars</font>** se define como un arreglo unidimensional que permite almacenar datos en un número arbitrario de observaciones, con ventajas más o menos similares a las que existen al comparar series de **<font color="mediumorchid">Pandas</font>** con arreglos unidimensionales de **<font color="mediumorchid">Numpy</font>**. Correspondientemente, una serie de **<font color="mediumorchid">Polars</font>** puede crearse desde cero utilizando la clase `pl.Series`, necesitándose para ello un único parámetro obligatorio llamado `values`, y que permiter definir los valores almacenados en la serie:

In [2]:
# Creamos una serie desde cero en Polars.
data = pl.Series(values=[0.25, 0.50, 0.75, 1.00])

In [3]:
# Mostramos la serie en pantalla.
data

""
f64
0.25
0.5
0.75
1.0


Notemos que, al imprimir una serie de **<font color="mediumorchid">Polars</font>**, la primera información de despoliega en la salida de nuestro código corresponde a la geometría de la misma, y que se observa directamente en el código anterior como `shape: (4,)`. Además, la impresión en pantalla tiene exactamente el mismo formato de **<font color="mediumorchid">IPython</font>** usado para mostrar DataFrames de **<font color="mediumorchid">Pandas</font>**, siendo el tipo de dato asociado a esta serie `f64`, y que se infiere por Python a partir de la data almacenada en la misma. Dicho tipo de dato puede consultarse por medio del atributo `dtype`:

In [4]:
# Tipo de dato asociado a la serie.
data.dtype

Float64

Como ocurre en las series de **<font color="mediumorchid">Pandas</font>**, es posible *bautizar* una serie usando el atributo `name`:

In [5]:
# Nombre de la serie.
data = pl.Series(values=[0.25, 0.50, 0.75, 1.00], name="series_name")

In [6]:
# Mostramos la serie en pantalla.
data

series_name
f64
0.25
0.5
0.75
1.0


Pero, a diferencia de las series de **<font color="mediumorchid">Pandas</font>**, no es posible cambiar este nombre mediante una asignación directa a partir del atributo `name` de la serie de **<font color="mediumorchid">Polars</font>**. De esta manera, el atributo `name` solamente nos servirá para chequear el nombre de la serie, no cambiarlo:

In [7]:
# Chequeamos el nombre de la serie.
data.name

'series_name'

In [8]:
# Si intentamos cambiar este nombre vía asignación, esto dará como resultado un error.
try:
    data.name = "asdf"
except AttributeError as e:
    print(e)

can't set attribute


Para cambiar el nombre asociado a una serie, basta con usar el método `rename()`:

In [9]:
# Renombramos la serie.
data.rename("asdf", in_place=True)

/var/folders/sn/hlgjzv2n1537dplx8bvvj1c80000gn/T/ipykernel_18105/3089101168.py:2: DeprecationWarning: the `in_place` parameter is deprecated and will be removed in a future version; note that renaming is a shallow-copy operation with essentially zero cost.
  data.rename("asdf", in_place=True)


asdf
f64
0.25
0.5
0.75
1.0


Notemos que, al igual que en **<font color="mediumorchid">Pandas</font>**, **<font color="mediumorchid">Polars</font>** nos permite modificar en el acto sus estructuras de datos, haciendo uso del parámetro Booleano `in_place`. Sin embargo, dicho parámetro se encuentra *deprecado* (en desuso), debido a que la operación de asignar una serie con otro nombre a otro objeto de Python es, esencialmente, una operación cuyo costo computacional es esencialmente igual a cero. Por esa razón, es recomendable que la operación anterior (y similares) se haga por medio de una asignación directa:

In [10]:
# Renombramos la serie vía asignación.
data = data.rename("asdf")

In [11]:
# Mostramos la serie en pantalla.
data

asdf
f64
0.25
0.5
0.75
1.0


Otra gran diferencia entre una serie de **<font color="mediumorchid">Polars</font>** y una serie de **<font color="mediumorchid">Pandas</font>** que podemos observar directamente al imprimir la serie anterior en pantalla, es que no existe un índice que nos permita seleccionar data a partir de la misma de manera completamente explícita. De este modo, la selección de datos en series de **<font color="mediumorchid">Polars</font>** es muy parecida a la que hacemos en arreglos unidimensionales de **<font color="mediumorchid">Numpy</font>**:

In [12]:
# Seleccionamos un único dato de la serie.
data[1]

0.5

In [13]:
# O varios mediante slicing.
data[1:3]

asdf
f64
0.5
0.75


No obstante, no podremos usar recursos de selección clásicos de **<font color="mediumorchid">Numpy</font>** (y, por extensión, de **<font color="mediumorchid">Pandas</font>**), como masking o fancy indexing, en series de **<font color="mediumorchid">Polars</font>**. Efectivamente, es posible construir máscaras en **<font color="mediumorchid">Polars</font>** mediante operadores de comparación. Por ejemplo:

In [14]:
# Una máscara sencilla.
mask = (data < 0.75)

In [15]:
# Imprimimos en pantalla la máscara anterior.
mask

asdf
bool
true
true
false
false


Vemos pues que la máscara anterior permite construir una serie de **<font color="mediumorchid">Polars</font>** únicamente con valores Booleanos. No obstante, no podemos hacer uso de ella para seleccionar datos a modo de filtro:

In [16]:
try:
    data[mask]
except ValueError as e:
    print(e)

Cannot __getitem__ on Series of dtype: 'Float64' with argument: 'shape: (4,)
Series: 'asdf' [bool]
[
	true
	true
	false
	false
]' of type: '<class 'polars.series.series.Series'>'.


**<font color="mediumorchid">Polars</font>** es claro al respecto en su mensaje de error: No es posible obtener ningún elemento de la misma a partir de la máscara anterior. La carencia de índices en una serie de **<font color="mediumorchid">Polars</font>** es en parte responsable por ésto, pero no es algo de lo que debamos preocuparnos. **<font color="mediumorchid">Polars</font>** nos provee de **métodos contextuales** que, entre otras transformaciones de interés, nos permiten construir filtros de datos tremendamente eficientes.

Las series de **<font color="mediumorchid">Polars</font>**, a diferencia de las series de **<font color="mediumorchid">Pandas</font>**, no pueden construirse directamente a partir de diccionarios de Python. Esto también se debe a que las series de **<font color="mediumorchid">Polars</font>** no cuentan con una correspondencia de tipo `(llave, valor)` que permita mapear los valores de la serie a un objeto explícitamente definido, precisamente por la carencia de índices. Sin embargo, como veremos más adelante, ésto tiene una razón bastante interesante de la cual podremos sacar un enorme provecho.

Otra característica que diferencia a las series de **<font color="mediumorchid">Polars</font>** de las de **<font color="mediumorchid">Pandas</font>**, es que éstas no disponen de un método `plot()` que nos permita construir gráficos a partir de los datos almacenados en ellas de forma rápida, aprovechando algún backend determinado (como **<font color="mediumorchid">Matplotlib</font>** en el caso de **<font color="mediumorchid">Pandas</font>**). Dependeremos enteramente de otras librerías para crear gráficos a partir de nuestros datos.

Las series de **<font color="mediumorchid">Polars</font>**, en cualquier caso, sí disponen de muchos métodos que nos permiten manipular la data almacenada en ellas. Vamos a ejemplificar ésto creando una nueva serie, a partir de un arreglo de datos normalmente distribuidos creado en **<font color="mediumorchid">Numpy</font>**:

In [17]:
# Semilla aleatoria fija.
rng = np.random.default_rng(42)

In [18]:
# Creamos un arreglo unidimensional con elementos normalmente distribudos.
arr_1 = rng.normal(loc=0, scale=1, size=5)

In [19]:
# Construimos una serie de Polars a partir de este arreglo.
s1 = pl.Series(values=arr_1, name="x1")

In [20]:
# Mostramos esta serie en pantalla.
s1

x1
f64
0.304717
-1.039984
0.750451
0.940565
-1.951035


Como ocurre con las series de **<font color="mediumorchid">Pandas</font>**, las series de **<font color="mediumorchid">Polars</font>** también disponen de métodos que permiten operar sobre los elementos de la misma, y en la mayoría de los casos, son métodos equivalentes a los disponibles en **<font color="mediumorchid">Pandas</font>**:

In [21]:
# Algunas operaciones básicas para la serie.
print(f"Media de los elementos de la serie = {s1.mean()}")
print(f"Desviación estándar de los elementos de la serie = {s1.std()}")
print(f"Varianza de los elementos de la serie = {s1.var()}")
print(f"Mínimo de los elementos de la serie = {s1.min()}")
print(f"Máximo de los elementos de la serie = {s1.max()}")

Media de los elementos de la serie = -0.1990572605884459
Desviación estándar de los elementos de la serie = 1.2480662808255583
Varianza de los elementos de la serie = 1.5576694413337413
Mínimo de los elementos de la serie = -1.9510351886538364
Máximo de los elementos de la serie = 0.9405647163912139


Las series de **<font color="mediumorchid">Polars</font>** también nos permiten recuperar los valores almacenados en las mismas en el formato de arreglos (unidimensionales) usando para ello el método `to_numpy()`. Sin embargo, al hacerlo, obtendremos una vista cuyo formato es `SeriesView`, propio de Arrow:

In [22]:
# Recuperamos los valores almacenados en la serie en un formato de arreglo.
s1.to_numpy()

array([ 0.30471708, -1.03998411,  0.7504512 ,  0.94056472, -1.95103519])

Incluso, podemos convertir series de **<font color="mediumorchid">Polars</font>** en series de **<font color="mediumorchid">Pandas</font>** usando el método `to_pandas()`. Esta transformación requiere de la instalación previa de la librería **<font color="mediumorchid">PyArrow</font>**:

In [23]:
# Convertimos la serie de Polars en una serie de Pandas.
s1.to_pandas()

0    0.304717
1   -1.039984
2    0.750451
3    0.940565
4   -1.951035
Name: x1, dtype: float64

Es posible unir una serie a otra serie en **<font color="mediumorchid">Polars</font>** usando el método `append()`:

In [24]:
# Creamos un arreglo unidimensional con elementos normalmente distribudos.
arr_2 = rng.normal(loc=1, scale=1, size=5)

In [25]:
# Construimos una serie de Polars a partir de este arreglo.
s2 = pl.Series(values=arr_2, name="x2")

In [26]:
# Unimos las series s1 y s2.
s3 = s1.append(s2)

In [27]:
# Mostramos dicha unión.
s3

x1
f64
0.304717
-1.039984
0.750451
0.940565
-1.951035
-0.30218
1.12784
0.683757
0.983199


Notemos que, al unir ambas series, se preserva el nombre asociado a la primera (sobre la cual usamos el método `append()`).

Existen otros métodos que nos permiten computar varios elementos de interés en una serie. Por ejemplo:

In [28]:
# Obtenemos la posición del valor mínimo asociado a la serie.
s3.arg_min()

4

In [29]:
# Obtenemos la posición del valor máximo asociado a la serie.
s3.arg_max()

6

In [30]:
# Obtenemos los k mayores elementos de la serie, ordenados de manera descendente.
s3.top_k(k=3)

x1
f64
1.12784
0.983199
0.940565


In [31]:
# Y los k menores, en esta ocasión ordenados de manera ascendente.
s3.bottom_k(k=3)

x1
f64
-1.951035
-1.039984
-0.30218


Por supuesto, podemos cambiar los tipos de datos asociados a una serie haciendo uso del método `cast()`:

In [32]:
# Cambiamos el tipo de dato de Float64 a Float32.
s3.cast(pl.Float32)

x1
f32
0.304717
-1.039984
0.750451
0.940565
-1.951035
-0.30218
1.12784
0.683757
0.983199


Y también podemos truncar los valores de una serie por medio del método `clip()`, donde los parámetros `lower_bound` y `upper_bound` permiten especificar los valores mínimos y máximos, respectivamente, que definen dicho truncamiento:

In [33]:
# Truncamos los valores de la serie entre 0 y 1.
s3.clip(0, 1)

x1
f64
0.304717
0.0
0.750451
0.940565
0.0
0.0
1.0
0.683757
0.983199


Existen muchos otros métodos aplicables a las series de **<font color="mediumorchid">Polars</font>**, los que iremos viendo a medida que vayamos avanzando con otros contenidos propios relativos al análisis de datos usando esta librería.

## DataFrames.
Al igual que en **<font color="mediumorchid">Pandas</font>**, en **<font color="mediumorchid">Polars</font>**, los DataFrames pueden definirse como una sucesión continua de series, las que constituyen una estructura tabular bidimensional de datos. Los DataFrames de **<font color="mediumorchid">Polars</font>** difieren esencialmente de los provistos por **<font color="mediumorchid">Pandas</font>** en el hecho de que sí poseen una correspondencia de tipo `(llave, valor)` explícita, a diferencia de las serie, pero únicamente para referenciar a las columnas del DataFrame (en realidad, para mapear cada serie individual que constituye el DataFrame completo).

Los DataFrames en **<font color="mediumorchid">Polars</font>** pueden construirse mediante la clase `pl.DataFrame`, usando el parámetro `data` para definir los valores que se almacenarán en esta estructura de datos:

In [34]:
# Creación de un DataFrame desde cero.
df = pl.DataFrame(data=rng.normal(loc=0, scale=1, size=(4, 4)))

In [35]:
# Mostramos este DataFrame en pantalla.
df

column_0,column_1,column_2,column_3
f64,f64,f64,f64
0.879398,0.777792,0.066031,1.127241
0.467509,-0.859292,0.368751,-0.958883
0.87845,-0.049926,-0.184862,-0.68093
1.222541,-0.154529,-0.428328,-0.352134


Notemos que, al mostrar este DataFrame haciendo uso de la interfaz provista por **<font color="mediumorchid">IPython</font>**, podemos observar que, al igual que con las series, lo primero que vemos es información relativa a su geometría, la cual es posible recuperar haciendo uso igualmente del atributo `shape`, y que nos informa que este DataFrame tiene 4 filas y 4 columnas:

In [36]:
# Geometría del DataFrame.
df.shape

(4, 4)

Cada columna, si no tiene un nombre definido en forma explícita, será nombrada como `column_0`, `column_1`, ..., `column_k`, para un DataFrame con un total de `k+1` columnas.

Si deseamos nombrar las columnas de un DataFrame explícitamente, debemos definir el **esquema** del mismo haciendo uso del parámetro `schema`. Con esquema nos referimos a la estructura inherente al DataFrame y que define la correspondencia `(nombre de la columna, tipo de dato)` propia de las estructuras de este tipo. Dada esta naturaleza, es común que los esquemas de un DataFrame se especifiquen por medio de diccionarios de Python:

In [37]:
# Construimos un DataFrame desde cero, definiendo el nombre de sus columnas a partir de su esquema.
df = pl.DataFrame(
    data=rng.normal(loc=0, scale=1, size=(4, 4)),
    schema={
        "col_1": pl.Float64, "col_2": pl.Float64, 
        "col_3": pl.Float64, "col_4": pl.Float64
    }
)

In [38]:
# Mostramos este DataFrame en pantalla.
df

col_1,col_2,col_3,col_4
f64,f64,f64,f64
0.532309,2.141648,0.615979,-0.824481
0.365444,-0.406415,1.128972,0.650593
0.412733,-0.512243,-0.113947,0.743254
0.430821,-0.813773,-0.840156,0.543154


**<font color="mediumorchid">Polars</font>** dispone de un sistema de impresión plana, prescindiendo de **<font color="mediumorchid">IPython</font>**, muy elegante, haciendo uso de la función nativa `print()`:

In [39]:
# ¡La impresión sin IPython de Polars es maravillosa!
print(df)

shape: (4, 4)
┌──────────┬───────────┬───────────┬───────────┐
│ col_1    ┆ col_2     ┆ col_3     ┆ col_4     │
│ ---      ┆ ---       ┆ ---       ┆ ---       │
│ f64      ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪═══════════╪═══════════╪═══════════╡
│ 0.532309 ┆ 2.141648  ┆ 0.615979  ┆ -0.824481 │
│ 0.365444 ┆ -0.406415 ┆ 1.128972  ┆ 0.650593  │
│ 0.412733 ┆ -0.512243 ┆ -0.113947 ┆ 0.743254  │
│ 0.430821 ┆ -0.813773 ┆ -0.840156 ┆ 0.543154  │
└──────────┴───────────┴───────────┴───────────┘


También es posible construir un DataFrame de **<font color="mediumorchid">Polars</font>** a partir de un esquema de tipo `(columna, valores asociados)`:

In [40]:
# Construimos un DataFrame a partir de un esquema de tipo diccionario.
df = pl.DataFrame(
    {
        "uniform": rng.uniform(low=-4, high=4, size=10),
        "normal": rng.normal(loc=0, scale=1, size=10),
        "poisson": rng.poisson(lam=1.0, size=10),
        "binomial": rng.binomial(n=100, p=0.4, size=10)
    }
)

In [41]:
# Mostramos este DataFrame en pantalla.
print(df)

shape: (10, 4)
┌───────────┬───────────┬─────────┬──────────┐
│ uniform   ┆ normal    ┆ poisson ┆ binomial │
│ ---       ┆ ---       ┆ ---     ┆ ---      │
│ f64       ┆ f64       ┆ i64     ┆ i64      │
╞═══════════╪═══════════╪═════════╪══════════╡
│ 2.658078  ┆ -1.457156 ┆ 1       ┆ 47       │
│ 2.438115  ┆ -0.319671 ┆ 1       ┆ 40       │
│ -0.900173 ┆ -0.470373 ┆ 0       ┆ 44       │
│ -1.693375 ┆ -0.638878 ┆ 1       ┆ 39       │
│ …         ┆ …         ┆ …       ┆ …        │
│ -2.400734 ┆ -0.865831 ┆ 0       ┆ 41       │
│ -3.941102 ┆ 0.968278  ┆ 0       ┆ 46       │
│ 2.295395  ┆ -1.68287  ┆ 0       ┆ 35       │
│ 1.318807  ┆ -0.334885 ┆ 2       ┆ 49       │
└───────────┴───────────┴─────────┴──────────┘


Una forma más *familiar* de construir DataFrames desde cero con respecto a lo que solemos hacer en **<font color="mediumorchid">Pandas</font>**, corresponde al uso de arreglos bidimensionales como input bajo el argumento `data`, y definir los nombres de las columnas por medio de listas bajo el argumento `schema`:

In [42]:
# Construimos un DataFrame definiendo los nombres de las columnas por medio de listas.
df = pl.DataFrame(
    data=rng.uniform(low=-1.0, high=1.0, size=(10, 10)),
    schema=[f"col_{j}" for j in range(0, 10)]
)

In [43]:
# Mostramos este DataFrame en pantalla.
print(df)

shape: (10, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ col_0     ┆ col_1     ┆ col_2     ┆ col_3     ┆ … ┆ col_6     ┆ col_7     ┆ col_8     ┆ col_9    │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -0.46826  ┆ -0.388087 ┆ -0.831111 ┆ 0.162122  ┆ … ┆ -0.539572 ┆ -0.48807  ┆ 0.037717  ┆ -0.75448 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 4        │
│ 0.938353  ┆ 0.158439  ┆ -0.168385 ┆ -0.30626  ┆ … ┆ -0.925175 ┆ 0.872087  ┆ -0.368142 ┆ 0.662225 │
│ 0.557502  ┆ -0.646454 ┆ -0.916772 ┆ 0.181831  ┆ … ┆ 0.109705  ┆ -0.670784 ┆ 0.544025  ┆ -0.69343 │
│           ┆           ┆           ┆           ┆   ┆           ┆          

Los DataFrames de **<font color="mediumorchid">Polars</font>** cuentan con métodos de inspección similares a los existentes en **<font color="mediumorchid">Pandas</font>**. Por ejemplo, los métodos `head()` y `tail()` permiten revisar, respectivamente, las primeras y últimas 5 columnas del DataFrame de interés:

In [44]:
# Mostramos las primeras 5 filas del DataFrame.
df.head()

col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.46826,-0.388087,-0.831111,0.162122,0.875653,-0.719502,-0.539572,-0.48807,0.037717,-0.754484
0.938353,0.158439,-0.168385,-0.30626,0.143456,0.108072,-0.925175,0.872087,-0.368142,0.662225
0.557502,-0.646454,-0.916772,0.181831,-0.053021,-0.782849,0.109705,-0.670784,0.544025,-0.693431
0.43378,0.713229,-0.012018,-0.954392,-0.466049,0.34448,-0.258155,-0.910179,0.323323,-0.641463
-0.101277,0.517039,-0.340278,0.917118,-0.336862,-0.437532,0.659579,-0.129806,-0.252685,0.198766


In [45]:
# Mostramos las últimas 5 filas del DataFrame.
df.tail()

col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.455517,0.438926,-0.710952,-0.035393,0.041345,0.318845,0.616503,0.984751,-0.811067,0.749124
-0.807218,-0.135814,-0.793194,0.56547,-0.122177,0.453989,-0.365722,0.783355,0.493579,-0.607131
0.805205,0.254618,0.175289,-0.83454,-0.956776,0.537295,0.905799,0.497216,-0.475079,-0.379353
-0.088447,0.168196,-0.658814,-0.026683,0.652584,-0.784518,-0.418164,0.781585,0.873626,0.55481
-0.595273,0.299693,0.85024,-0.018586,0.792322,0.832024,0.030114,0.786893,-0.518059,0.943653


También es posible recuperar algunas filas al azar de un DataFrame de **<font color="mediumorchid">Polars</font>** por medio del método `sample()`:

In [46]:
# Extraemos dos filas al azar de nuestro DataFrame.
df.sample(n=2)

col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.807218,-0.135814,-0.793194,0.56547,-0.122177,0.453989,-0.365722,0.783355,0.493579,-0.607131
0.938353,0.158439,-0.168385,-0.30626,0.143456,0.108072,-0.925175,0.872087,-0.368142,0.662225


Siempre podemos mantener la reproducibilidad de los muestreos de filas en un DataFrame por medio del parámetro `seed`:

In [47]:
# Extraemos dos filas al azar de nuestro DataFrame, con una semilla aleatoria fija.
df.sample(n=2, seed=42)

col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.595273,0.299693,0.85024,-0.018586,0.792322,0.832024,0.030114,0.786893,-0.518059,0.943653
0.805205,0.254618,0.175289,-0.83454,-0.956776,0.537295,0.905799,0.497216,-0.475079,-0.379353


Podemos seleccionar cualquier columna en un DataFrame de **<font color="mediumorchid">Polars</font>** haciendo uso del mismo esquema de indexación sencillo usado en **<font color="mediumorchid">Pandas</font>**, nombrando la columna de interés por medio de corchetes, o bien, listas anidadas (fancy indexing):

In [48]:
# Seleccionamos una columna del DataFrame.
df["col_3"]

col_3
f64
0.162122
-0.30626
0.181831
-0.954392
0.917118
-0.035393
0.56547
-0.83454
-0.026683


In [49]:
# Seleccionamos más de una columna mediante fancy indexing.
df[["col_1", "col_4", "col_7"]]

col_1,col_4,col_7
f64,f64,f64
-0.388087,0.875653,-0.48807
0.158439,0.143456,0.872087
-0.646454,-0.053021,-0.670784
0.713229,-0.466049,-0.910179
0.517039,-0.336862,-0.129806
0.438926,0.041345,0.984751
-0.135814,-0.122177,0.783355
0.254618,-0.956776,0.497216
0.168196,0.652584,0.781585


También, al igual que en **<font color="mediumorchid">Pandas</font>**, en **<font color="mediumorchid">Polars</font>** disponemos de un método `describe()` para obtener un resumen estadístico de las columnas que constituyen el DataFrame:

In [50]:
# Resumen estadístico sencillo del DataFrame.
df.describe()

describe,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.021885,0.137978,-0.340599,-0.034931,0.057047,-0.01297,-0.018509,0.250705,-0.015276,0.003272
"""std""",0.621275,0.418104,0.561516,0.568764,0.584426,0.610539,0.593245,0.724655,0.553547,0.684065
"""min""",-0.807218,-0.646454,-0.916772,-0.954392,-0.956776,-0.784518,-0.925175,-0.910179,-0.811067,-0.754484
"""max""",0.938353,0.713229,0.85024,0.917118,0.875653,0.832024,0.905799,0.984751,0.873626,0.943653
"""median""",-0.094862,0.211407,-0.499546,-0.022635,-0.005838,0.213459,-0.114021,0.639401,-0.107484,-0.090294
"""25%""",-0.46826,-0.135814,-0.793194,-0.30626,-0.336862,-0.719502,-0.418164,-0.48807,-0.475079,-0.641463
"""75%""",0.557502,0.438926,-0.012018,0.181831,0.652584,0.453989,0.616503,0.786893,0.493579,0.662225
